In [165]:
#xgboost algorithm

In [166]:
!pip install pandas
!pip install matplotlib
!pip install statsmodels
!pip install scikit-learn
!pip install numpy seaborn
!pip install numpy pandas matplotlib seaborn
!pip install --upgrade scikit-learn
!pip install xgboost

In [167]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.simplefilter('ignore')
import os
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn import model_selection
from sklearn.model_selection import train_test_split

from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [168]:
pd.options.display.max_columns=100

In [169]:
#Load The data
df_train = pd.read_csv('/Users/unmeshmhatre/Desktop/Data_Analysis/24-09-2023/train.csv')
df_features = pd.read_csv('/Users/unmeshmhatre/Desktop/Data_Analysis/24-09-2023/features.csv')

In [170]:
# To convert Date to datetime
df_train["Date"] = pd.to_datetime(df_train["Date"])
df_features["Date"] = pd.to_datetime(df_features["Date"])

In [171]:
df_train.shape , df_features.shape

((421570, 5), (8190, 12))

In [172]:
df_features

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False
...,...,...,...,...,...,...,...,...,...,...,...,...
8185,45,2013-06-28,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN,False
8186,45,2013-07-05,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN,False
8187,45,2013-07-12,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN,False
8188,45,2013-07-19,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,NaN,NaN,False


In [173]:
df_train

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


In [174]:
df_train.isna().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64

In [175]:
df_train.head(10)

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
5,1,1,2010-03-12,21043.39,False
6,1,1,2010-03-19,22136.64,False
7,1,1,2010-03-26,26229.21,False
8,1,1,2010-04-02,57258.43,False
9,1,1,2010-04-09,42960.91,False


In [176]:
df_features.isna().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

In [177]:
train_data = pd.merge(df_train, df_features, on=['Date','Store'], how='inner')

In [178]:
train_data.tail(30)

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
421540,45,41,2012-10-26,1954.67,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421541,45,42,2012-10-26,4894.78,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421542,45,44,2012-10-26,3144.09,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421543,45,46,2012-10-26,13329.25,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421544,45,52,2012-10-26,1104.16,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421545,45,54,2012-10-26,23.92,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421546,45,55,2012-10-26,3458.17,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421547,45,56,2012-10-26,937.72,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421548,45,58,2012-10-26,275.00,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421549,45,59,2012-10-26,191.98,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False


In [179]:
train_data.isna().sum()

Store                0
Dept                 0
Date                 0
Weekly_Sales         0
IsHoliday_x          0
Temperature          0
Fuel_Price           0
MarkDown1       270889
MarkDown2       310322
MarkDown3       284479
MarkDown4       286603
MarkDown5       270138
CPI                  0
Unemployment         0
IsHoliday_y          0
dtype: int64

In [180]:
cols_to_drop = ["IsHoliday_y","MarkDown1", "MarkDown2", "MarkDown3","MarkDown4","MarkDown5","CPI","Temperature","Fuel_Price","Unemployment"]
train_data.drop(columns=cols_to_drop, inplace=True)

In [181]:
train_data.head(10)

,Store,Dept,Date,Weekly_Sales,IsHoliday_x
0,1,1,2010-02-05,24924.50,False
1,1,2,2010-02-05,50605.27,False
2,1,3,2010-02-05,13740.12,False
3,1,4,2010-02-05,39954.04,False
4,1,5,2010-02-05,32229.38,False
5,1,6,2010-02-05,5749.03,False
6,1,7,2010-02-05,21084.08,False
7,1,8,2010-02-05,40129.01,False
8,1,9,2010-02-05,16930.99,False
9,1,10,2010-02-05,30721.50,False


In [182]:
train_data.shape

(421570, 5)

In [183]:
duplicates = train_data[train_data.duplicated()]

# To Display the duplicate rows from dataset
if not duplicates.empty:
    print(duplicates)
else:
    print("No duplicate rows found.")

# Removing the duplicate rows
train_data = train_data.drop_duplicates()

No duplicate rows found.


In [184]:
# check for the outliers
forecasting_column = 'Weekly_Sales'

Data_1 = np.percentile(train_data[forecasting_column], 25)
Data_2 = np.percentile(train_data[forecasting_column], 75)


range_multiplier = 1.5 
Interquartile_Range = Data_2 - Data_1

lower_range = Data_1 - range_multiplier * Interquartile_Range
upper_range = Data_2 + range_multiplier * Interquartile_Range


detect_outliers = train_data[(train_data[forecasting_column] < lower_range) | (train_data[forecasting_column] > upper_range)]

In [185]:
detect_outliers

,Store,Dept,Date,Weekly_Sales,IsHoliday_x
1,1,2,2010-02-05,50605.27,False
36,1,38,2010-02-05,115564.35,False
37,1,40,2010-02-05,66780.63,False
56,1,72,2010-02-05,98499.12,False
65,1,90,2010-02-05,76419.47,False
...,...,...,...,...,...
421497,45,92,2012-10-19,48108.38,False
421500,45,95,2012-10-19,48434.97,False
421538,45,38,2012-10-26,47799.07,False
421564,45,92,2012-10-26,54608.75,False


In [186]:
train_data = train_data[~train_data.index.isin(detect_outliers.index)]

In [187]:
train_data

,Store,Dept,Date,Weekly_Sales,IsHoliday_x
0,1,1,2010-02-05,24924.50,False
2,1,3,2010-02-05,13740.12,False
3,1,4,2010-02-05,39954.04,False
4,1,5,2010-02-05,32229.38,False
5,1,6,2010-02-05,5749.03,False
...,...,...,...,...,...
421563,45,91,2012-10-26,16330.84,False
421565,45,93,2012-10-26,2487.80,False
421566,45,94,2012-10-26,5203.31,False
421568,45,97,2012-10-26,6817.48,False


In [188]:
cols_to_drop = ["Store","Dept"]
train_data.drop(columns=cols_to_drop, inplace=True)

In [189]:
train_data

,Date,Weekly_Sales,IsHoliday_x
0,2010-02-05,24924.50,False
2,2010-02-05,13740.12,False
3,2010-02-05,39954.04,False
4,2010-02-05,32229.38,False
5,2010-02-05,5749.03,False
...,...,...,...
421563,2012-10-26,16330.84,False
421565,2012-10-26,2487.80,False
421566,2012-10-26,5203.31,False
421568,2012-10-26,6817.48,False


In [190]:
train_data.rename(columns={'Date':'ds','Weekly_Sales':'y','IsHoliday_x':'IsHoliday' }, inplace=True)

In [191]:
train_data

,ds,y,IsHoliday
0,2010-02-05,24924.50,False
2,2010-02-05,13740.12,False
3,2010-02-05,39954.04,False
4,2010-02-05,32229.38,False
5,2010-02-05,5749.03,False
...,...,...,...
421563,2012-10-26,16330.84,False
421565,2012-10-26,2487.80,False
421566,2012-10-26,5203.31,False
421568,2012-10-26,6817.48,False


In [192]:
df_xgboost = train_data.copy()
df_xgboost

,ds,y,IsHoliday
0,2010-02-05,24924.50,False
2,2010-02-05,13740.12,False
3,2010-02-05,39954.04,False
4,2010-02-05,32229.38,False
5,2010-02-05,5749.03,False
...,...,...,...
421563,2012-10-26,16330.84,False
421565,2012-10-26,2487.80,False
421566,2012-10-26,5203.31,False
421568,2012-10-26,6817.48,False


In [193]:
# Converting categorical columns to numerical columns and removing the old columns
mappings = {0:0, "False": 0, "True": 1}
df_xgboost["IsHoliday_cat"] = df_xgboost["IsHoliday"].map(mappings)
df_xgboost = df_xgboost.drop(["IsHoliday"], axis = 1)
df_xgboost.set_index('ds', inplace=True)


In [194]:
df_xgboost

,y,IsHoliday_cat
ds,,
2010-02-05,24924.50,0.0
2010-02-05,13740.12,0.0
2010-02-05,39954.04,0.0
2010-02-05,32229.38,0.0
2010-02-05,5749.03,0.0
...,...,...
2012-10-26,16330.84,0.0
2012-10-26,2487.80,0.0
2012-10-26,5203.31,0.0


In [195]:
#resample data on weekly basis 
resampled_df = df_xgboost.resample('W').mean()

In [196]:
resampled_df

,y,IsHoliday_cat
ds,,
2010-02-07,10371.766994,0.0
2010-02-14,10945.257698,NaN
2010-02-21,10730.564159,0.0
2010-02-28,9917.680665,0.0
2010-03-07,10313.762879,0.0
...,...,...
2012-09-30,9994.956797,0.0
2012-10-07,10252.912027,0.0
2012-10-14,10409.566168,0.0


In [197]:
resampled_df.isna()

,y,IsHoliday_cat
ds,,
2010-02-07,False,False
2010-02-14,False,True
2010-02-21,False,False
2010-02-28,False,False
2010-03-07,False,False
...,...,...
2012-09-30,False,False
2012-10-07,False,False
2012-10-14,False,False


In [198]:
resampled_df = resampled_df.dropna()

In [199]:
resampled_df

,y,IsHoliday_cat
ds,,
2010-02-07,10371.766994,0.0
2010-02-21,10730.564159,0.0
2010-02-28,9917.680665,0.0
2010-03-07,10313.762879,0.0
2010-03-14,10389.785914,0.0
...,...,...
2012-09-30,9994.956797,0.0
2012-10-07,10252.912027,0.0
2012-10-14,10409.566168,0.0


In [200]:
# split the Data based on date
split_date = pd.to_datetime("2012-02-01")

In [201]:
# Filter data for training based on the split date
X_train = resampled_df.loc[resampled_df.index < split_date].drop("y", axis=1)
y_train = resampled_df.loc[resampled_df.index < split_date]["y"]

In [202]:
# Filter data for testing based on the split date
X_test = resampled_df.loc[resampled_df.index >= split_date].drop("y", axis=1)
y_test = resampled_df.loc[resampled_df.index >= split_date]["y"]

In [203]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (96, 1)
y_train shape: (96,)
X_test shape: (37, 1)
y_test shape: (37,)


In [204]:
# for tuning the parameters
tune_parameters = {"max_depth":6, "booster": "gbtree", "eta":0.3, "objective":"reg:linear","subsample": 0.9, "colsample_bytree": 0.7,"seed": None}
data_train = xgb.DMatrix(X_train, y_train)

In [205]:
data_test = xgb.DMatrix(X_test, y_test)
W1 = [(data_train, 'train'), (data_test, 'eval')]

In [206]:
#To train xgboost model
xgboost = xgb.train(tune_parameters, data_train, 100, evals=W1,early_stopping_rounds= 100, verbose_eval=True)

[0]	train-rmse:659.14619	eval-rmse:327.86110
[1]	train-rmse:659.11687	eval-rmse:326.95543
[2]	train-rmse:659.25825	eval-rmse:325.39379
[3]	train-rmse:659.38036	eval-rmse:324.96245
[4]	train-rmse:659.31086	eval-rmse:325.18193
[5]	train-rmse:659.12044	eval-rmse:326.67436
[6]	train-rmse:659.11935	eval-rmse:326.72138
[7]	train-rmse:659.16510	eval-rmse:325.97725
[8]	train-rmse:659.16732	eval-rmse:325.95718
[9]	train-rmse:659.12212	eval-rmse:327.32801
[10]	train-rmse:659.11714	eval-rmse:327.04311
[11]	train-rmse:659.19546	eval-rmse:325.73905
[12]	train-rmse:659.14880	eval-rmse:326.14611
[13]	train-rmse:659.13988	eval-rmse:326.26105
[14]	train-rmse:659.12216	eval-rmse:326.61402
[15]	train-rmse:659.12857	eval-rmse:326.45281
[16]	train-rmse:659.12527	eval-rmse:327.42881
[17]	train-rmse:659.15035	eval-rmse:327.92710
[18]	train-rmse:659.14422	eval-rmse:327.82838
[19]	train-rmse:659.24003	eval-rmse:328.92926
[20]	train-rmse:659.25197	eval-rmse:329.03441
[21]	train-rmse:659.18099	eval-rmse:328.3328

In [207]:
predictions = xgboost.predict(data_test)

In [208]:
predictions

array([10551.34, 10551.34, 10551.34, 10551.34, 10551.34, 10551.34,
       10551.34, 10551.34, 10551.34, 10551.34, 10551.34, 10551.34,
       10551.34, 10551.34, 10551.34, 10551.34, 10551.34, 10551.34,
       10551.34, 10551.34, 10551.34, 10551.34, 10551.34, 10551.34,
       10551.34, 10551.34, 10551.34, 10551.34, 10551.34, 10551.34,
       10551.34, 10551.34, 10551.34, 10551.34, 10551.34, 10551.34,
       10551.34], dtype=float32)

In [209]:
rmse_xgboost = np.sqrt(mean_squared_error(y_test, predictions))

In [210]:
rmse_xgboost

327.614189151229

In [211]:
Data = pd.DataFrame()

In [212]:
Data['Actual Sales'] = y_test
Data['Predicted Sales'] = predictions

In [213]:
Data

,Actual Sales,Predicted Sales
ds,,
2012-02-05,9685.453986,10551.339844
2012-02-19,10671.778309,10551.339844
2012-02-26,10130.651289,10551.339844
2012-03-04,10037.057146,10551.339844
2012-03-11,10352.812416,10551.339844
2012-03-18,10519.416761,10551.339844
2012-03-25,10607.530871,10551.339844
2012-04-01,10481.808055,10551.339844
2012-04-08,11080.157876,10551.339844


In [216]:
print("Root Mean Squared Error for the XGBoost Model:",rmse_xgboost)

Root Mean Squared Error for the XGBoost Model: 327.614189151229
